* to get the partitons
* converting pandas dataframe into Spark
* Spark SQL
* Global view
* UDF--- Lambda
* Structtype
* Maptype
* Arraytype
* Row
* Locate
* SelectExpr
* drop_duplicates
* No of rows and columns
* Sorting API
* Set
* Join(cross,leftanti,leftsemi,cross)
* Aggregate(Summary,

In [1]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql.functions import udf,col,length,expr,lit,lower,sumDistinct,sum,round,count,countDistinct,max,min
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Training").getOrCreate()

## to get the partitons 

In [2]:
dd=pd.DataFrame(data={"Name":["a","b","c"],"Age":[10,20,30]})
df_emp=spark.createDataFrame(dd)
df_emp.show()

+----+---+
|Name|Age|
+----+---+
|   a| 10|
|   b| 20|
|   c| 30|
+----+---+



In [3]:
df_emp.rdd.getNumPartitions()

4

## converting pandas dataframe into Spark

In [4]:
dd=pd.DataFrame(data={"Name":["a","b","c"],"Age":[10,20,30]})
dd

,Name,Age
0,a,10
1,b,20
2,c,30


In [5]:
df_emp=spark.createDataFrame(dd)
df_emp.show()


+----+---+
|Name|Age|
+----+---+
|   a| 10|
|   b| 20|
|   c| 30|
+----+---+



## Spark SQL

In [6]:

df_emp.createOrReplaceTempView("emp")

In [7]:

spark.sql("""select * from emp where Age >20""").show()

+----+---+
|Name|Age|
+----+---+
|   c| 30|
+----+---+



In [8]:
dg=pd.DataFrame(data={"Name":["a1","b1","c1"],"Age":[10,20,30],"Sex":["M","F","M"]})
dep=spark.createDataFrame(dg)
dep.show()

+----+---+---+
|Name|Age|Sex|
+----+---+---+
|  a1| 10|  M|
|  b1| 20|  F|
|  c1| 30|  M|
+----+---+---+



## Global view

In [9]:
dep.createOrReplaceGlobalTempView("jon")

In [10]:
spark.sql(""" select * from global_temp.jon where Age>20 and Sex='M'  """).show()

+----+---+---+
|Name|Age|Sex|
+----+---+---+
|  c1| 30|  M|
+----+---+---+



## table

In [11]:
df=spark.table("emp")
df.show()

+----+---+
|Name|Age|
+----+---+
|   a| 10|
|   b| 20|
|   c| 30|
+----+---+



## reading text file

In [12]:
df=spark.read.text("tst.txt",)
df.show()

+---------------+
|          value|
+---------------+
|  Name Age city|
|Nitish 32 delhi|
|  rakesh 33 goa|
|   king 35 agra|
+---------------+



# UDF

In [13]:
s="nitish bhardwaj king is back"
def cps(st):
    s=st.split(" ")
    cp=""
    for i in s:
        cp=cp+i[0].upper()+i[1:]+" " 
    return cp

In [14]:
cps(s)

'Nitish Bhardwaj King Is Back '

In [15]:
dd=pd.DataFrame(data={"Name":["chegiz khan","bryan fury","jin khazma"],"Age":[10,20,30]})
df=spark.createDataFrame(dd)
df.show()

+-----------+---+
|       Name|Age|
+-----------+---+
|chegiz khan| 10|
| bryan fury| 20|
| jin khazma| 30|
+-----------+---+



# UDF in PySpark

In [2]:
from pyspark.sql.functions import udf,col,length
from pyspark.sql.types import IntegerType

In [17]:
@udf()
def cps(st):
    s=st.split(" ")
    cp=""
    for i in s:
        cp=cp+i[0].upper()+i[1:]+" " 
    return cp

In [18]:
df.select(col("*"),cps(df.Name)).show()

+-----------+---+------------+
|       Name|Age|   cps(Name)|
+-----------+---+------------+
|chegiz khan| 10|Chegiz Khan |
| bryan fury| 20| Bryan Fury |
| jin khazma| 30| Jin Khazma |
+-----------+---+------------+



## lambda in spark/Pandas

In [40]:
dd["size"]=dd.apply(lambda x:len(x["Name"]),axis=1)
dd


,Name,Age,size
0,chegiz khan,10,11
1,bryan fury,20,10
2,jin khazma,30,10


In [66]:
df.show()

+-----------+---+
|       Name|Age|
+-----------+---+
|chegiz khan| 10|
| bryan fury| 20|
| jin khazma| 30|
+-----------+---+



In [49]:
df.select(col("*"),length(df.Name)).show()

+-----------+---+------------+
|       Name|Age|length(Name)|
+-----------+---+------------+
|chegiz khan| 10|          11|
| bryan fury| 20|          10|
| jin khazma| 30|          10|
+-----------+---+------------+



In [57]:
spp=udf(lambda x:length(x),IntegerType())
spp

<function __main__.<lambda>(x)>

In [20]:
df.select(length(df.Name)).show()

+------------+
|length(Name)|
+------------+
|          11|
|          10|
|          10|
+------------+



### to check current database

In [68]:
spark.catalog.currentDatabase()

'default'

In [61]:
spark.catalog.setCurrentDatabase("default")

In [62]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/C:/Users/nitis/Downloads/Documents/PySpark/spark-warehouse')]

# Structtype

In [3]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DataType,MapType,ArrayType

In [5]:
schema=StructType([
    StructField("Name",StringType(),True),
    StructField("ID",IntegerType(),True),
])

In [7]:
data=[("James",1),
     ("Robet",2),
     ("John",3)]
df=spark.createDataFrame(data=data,schema=schema)
df.show()

+-----+---+
| Name| ID|
+-----+---+
|James|  1|
|Robet|  2|
| John|  3|
+-----+---+



In [8]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- ID: integer (nullable = true)



## MapType

In [10]:
schema=StructType([
    StructField("Name",StringType(),True),
    StructField("Properties",MapType(StringType(),StringType()),True),
])

In [12]:
data=[("James",{"hair":"black","eye":"Brown"}),
     ("Robet",{"hair":"White","eye":"Black"}),
     ("John",{"hair":"Blonde","eye":"Blue"})]
df=spark.createDataFrame(data=data,schema=schema)
df.show(truncate=False)

+-----+-----------------------------+
|Name |Properties                   |
+-----+-----------------------------+
|James|{eye -> Brown, hair -> black}|
|Robet|{eye -> Black, hair -> White}|
|John |{eye -> Blue, hair -> Blonde}|
+-----+-----------------------------+



In [18]:
df.select(col("Properties")["eye"]).show(truncate=False)

+---------------+
|Properties[eye]|
+---------------+
|Brown          |
|Black          |
|Blue           |
+---------------+



# ArrayType

In [23]:
schema=StructType([
    StructField("Name",StringType(),True),
    StructField("PhoneNum",ArrayType(IntegerType()),True),
])

In [24]:
data=[("James",[123,900]),
     ("Robet",[555,999]),
     ("John",[888,100])]
df=spark.createDataFrame(data=data,schema=schema)
df.show(truncate=False)

+-----+----------+
|Name |PhoneNum  |
+-----+----------+
|James|[123, 900]|
|Robet|[555, 999]|
|John |[888, 100]|
+-----+----------+



In [25]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- PhoneNum: array (nullable = true)
 |    |-- element: integer (containsNull = true)



# Row

In [4]:
from pyspark.sql import Row

In [5]:
row=Row(Name="Alex",Age=35)
row.Name

'Alex'

In [38]:
"Name" in row

True

In [39]:
"Alex" in row.Name

True

In [40]:
"King" in row.Name

False

### creating rdd from row

In [28]:
lst=[Row(Name="Alex",Age=35),Row(Name="Jin",Age=23),Row(Name="King",Age=33),Row(Name="John",Age=34)]
rdd=spark.sparkContext.parallelize(lst)
for i in rdd.take(len(lst)):
    print(i)

Row(Name='Alex', Age=35)
Row(Name='Jin', Age=23)
Row(Name='King', Age=33)
Row(Name='John', Age=34)


In [30]:
for i in rdd.take(len(lst)):
    print(i.Name+" "+str(i.Age))

Alex 35
Jin 23
King 33
John 34


## create dataframe using rows

In [31]:
dt=spark.createDataFrame(lst)
print(dt.printSchema())
print("-"*100)
print(dt.show())

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)

None
----------------------------------------------------------------------------------------------------
+----+---+
|Name|Age|
+----+---+
|Alex| 35|
| Jin| 23|
|King| 33|
|John| 34|
+----+---+

None


In [34]:
dt=dt.withColumn("Age",col("Age").astype("int"))
dt.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)



## Custom Rows like Class

In [35]:
person=Row("Name","Age")
p1=person("John",33)
p2=person("Aman",25)
p1.Name

'John'

### count in rows

In [41]:
p=Row(Name="Alex",Age=35,username="Alex")
p.count("Alex")

2

### index in row
* return first occurence

In [42]:
p

Row(Name='Alex', Age=35, username='Alex')

In [43]:
p.index(35)

1

### asdict in rows

In [44]:
p.asDict()

{'Name': 'Alex', 'Age': 35, 'username': 'Alex'}

## Alternate of where

In [45]:
dt.show()

+----+---+
|Name|Age|
+----+---+
|Alex| 35|
| Jin| 23|
|King| 33|
|John| 34|
+----+---+



In [47]:
dt[dt.Age==35].show()

+----+---+
|Name|Age|
+----+---+
|Alex| 35|
+----+---+



In [48]:
dt.where(dt.Age==35).show()

+----+---+
|Name|Age|
+----+---+
|Alex| 35|
+----+---+



In [53]:
dt.select(length(dt.Name)).show()

+------------+
|length(Name)|
+------------+
|           4|
|           3|
|           4|
|           4|
+------------+



In [97]:
#dt.withColumn(col("*"),dt.Name.substr(2,int(length(dt.Name))).alias("test")).show()

dt.withColumn("Vale", dt.Name.substr(lit(2), length("Name"))).show()

+----+---+----+
|Name|Age|Vale|
+----+---+----+
|Alex| 35| lex|
| Jin| 23|  in|
|King| 33| ing|
|John| 34| ohn|
+----+---+----+



## using collect

In [71]:
[i.Name for i in dt.select(dt["Name"]).collect()]

['Alex', 'Jin', 'King', 'John']

# ETL

In [3]:
df=spark.read.load('orders/part-00000',format="csv",schema='order_id int,order_date timestamp,order_customer_id int, \
                   order_status string')
df.show()

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
|       6|2013-07-25 00:00:00|             7130|       COMPLETE|
|       7|2013-07-25 00:00:00|             4530|       COMPLETE|
|       8|2013-07-25 00:00:00|             2911|     PROCESSING|
|       9|2013-07-25 00:00:00|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|             1837|         CLOSED|
|      13|2013-07-25 00:0

## lower-- Convert to lower case

In [11]:
df.select(lower("order_status")).show(5)

+-------------------+
|lower(order_status)|
+-------------------+
|             closed|
|    pending_payment|
|           complete|
|             closed|
|           complete|
+-------------------+
only showing top 5 rows



## select expr--variant of select that accept SQL statemets

In [16]:
df.select(df.order_date.substr(1,4).alias("SubString")).show(5)

+---------+
|SubString|
+---------+
|     2013|
|     2013|
|     2013|
|     2013|
|     2013|
+---------+
only showing top 5 rows



In [23]:
df.selectExpr("substring(order_date,1,4) as new_order").show(5)

+---------+
|new_order|
+---------+
|     2013|
|     2013|
|     2013|
|     2013|
|     2013|
+---------+
only showing top 5 rows



## Stack-- Used to unpivot the data, but not in Pyspark thats why we use stack

In [6]:
from  pyspark.sql.functions import stack


ImportError: cannot import name 'stack' from 'pyspark.sql.functions' (D:\Software_install\Anaconda\envs\SPARK_HOME\lib\site-packages\pyspark\sql\functions.py)

In [28]:
nd=spark.range(1)
nd.show()

+---+
| id|
+---+
|  0|
+---+



In [29]:
nd.selectExpr("stack(3, 1,2,3,4,5,6)").show()

+----+----+
|col0|col1|
+----+----+
|   1|   2|
|   3|   4|
|   5|   6|
+----+----+



* here value 1,2,3,4,5,6 is split into 3 rows as we have stack(3,-----)

# Locate

In [2]:
import pyspark.sql.functions as F

In [5]:
df.select(F.locate("_",df.order_status).alias("locate")).show(3)

+------+
|locate|
+------+
|     0|
|     8|
|     0|
+------+
only showing top 3 rows



## drop_duplicates

In [31]:
df.drop_duplicates(subset=["order_status"]).show()

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|      46|2013-07-25 00:00:00|             1549|        ON_HOLD|
|      11|2013-07-25 00:00:00|              918| PAYMENT_REVIEW|
|       8|2013-07-25 00:00:00|             2911|     PROCESSING|
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|      69|2013-07-25 00:00:00|             2821|SUSPECTED_FRAUD|
|      21|2013-07-25 00:00:00|             2711|        PENDING|
|      50|2013-07-25 00:00:00|             5225|       CANCELED|
+--------+-------------------+-----------------+---------------+



## diff between select and where/filter API

In [38]:
df.select((col("order_id")>10) & (col("order_id")<20)).show(11)

+-------------------------------------+
|((order_id > 10) AND (order_id < 20))|
+-------------------------------------+
|                                false|
|                                false|
|                                false|
|                                false|
|                                false|
|                                false|
|                                false|
|                                false|
|                                false|
|                                false|
|                                 true|
+-------------------------------------+
only showing top 11 rows



In [39]:
df.where((col("order_id")>10) & (col("order_id")<20)).show()

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|      11|2013-07-25 00:00:00|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|             1837|         CLOSED|
|      13|2013-07-25 00:00:00|             9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|             9842|     PROCESSING|
|      15|2013-07-25 00:00:00|             2568|       COMPLETE|
|      16|2013-07-25 00:00:00|             7276|PENDING_PAYMENT|
|      17|2013-07-25 00:00:00|             2667|       COMPLETE|
|      18|2013-07-25 00:00:00|             1205|         CLOSED|
|      19|2013-07-25 00:00:00|             9488|PENDING_PAYMENT|
+--------+-------------------+-----------------+---------------+



## ISIN

In [40]:
df.where(col("order_status").isin("CLOSED","COMPLETE")).show()

+--------+-------------------+-----------------+------------+
|order_id|         order_date|order_customer_id|order_status|
+--------+-------------------+-----------------+------------+
|       1|2013-07-25 00:00:00|            11599|      CLOSED|
|       3|2013-07-25 00:00:00|            12111|    COMPLETE|
|       4|2013-07-25 00:00:00|             8827|      CLOSED|
|       5|2013-07-25 00:00:00|            11318|    COMPLETE|
|       6|2013-07-25 00:00:00|             7130|    COMPLETE|
|       7|2013-07-25 00:00:00|             4530|    COMPLETE|
|      12|2013-07-25 00:00:00|             1837|      CLOSED|
|      15|2013-07-25 00:00:00|             2568|    COMPLETE|
|      17|2013-07-25 00:00:00|             2667|    COMPLETE|
|      18|2013-07-25 00:00:00|             1205|      CLOSED|
|      22|2013-07-25 00:00:00|              333|    COMPLETE|
|      24|2013-07-25 00:00:00|            11441|      CLOSED|
|      25|2013-07-25 00:00:00|             9503|      CLOSED|
|      2

## No of rows and columns

In [49]:
rows=df.count()
print(f"DataFrame Rows count : {rows}")
cols=len(df.columns)
print(f"No of columns are :{cols}")

DataFrame Rows count : 68883
No of columns are :4


## Sorting API
* we can use order_by() or sort()

In [53]:
df.sort(col("order_customer_id").desc()).show(10)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|   41643|2014-04-08 00:00:00|            12435|        PENDING|
|   61629|2013-12-21 00:00:00|            12435|       CANCELED|
|    6160|2013-09-02 00:00:00|            12434|       COMPLETE|
|    4799|2013-08-23 00:00:00|            12434|PENDING_PAYMENT|
|   42915|2014-04-16 00:00:00|            12434|       COMPLETE|
|    1868|2013-08-03 00:00:00|            12434|         CLOSED|
|   13544|2013-10-16 00:00:00|            12434|        PENDING|
|   51800|2014-06-14 00:00:00|            12434|        ON_HOLD|
|   61777|2013-12-26 00:00:00|            12434|       COMPLETE|
|    5303|2013-08-26 00:00:00|            12434|        PENDING|
+--------+-------------------+-----------------+---------------+
only showing top 10 rows



In [52]:
df.orderBy(col("order_customer_id")).show(10)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|   22945|2013-12-13 00:00:00|                1|       COMPLETE|
|   15192|2013-10-29 00:00:00|                2|PENDING_PAYMENT|
|   33865|2014-02-18 00:00:00|                2|       COMPLETE|
|   57963|2013-08-02 00:00:00|                2|        ON_HOLD|
|   67863|2013-11-30 00:00:00|                2|       COMPLETE|
|   35158|2014-02-26 00:00:00|                3|       COMPLETE|
|   22646|2013-12-11 00:00:00|                3|       COMPLETE|
|   57617|2014-07-24 00:00:00|                3|       COMPLETE|
|   23662|2013-12-19 00:00:00|                3|       COMPLETE|
|   56178|2014-07-15 00:00:00|                3|        PENDING|
+--------+-------------------+-----------------+---------------+
only showing top 10 rows



## sorting multiple columns

In [60]:
df.sort(col("order_customer_id"),col("order_status")).show(10)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|   22945|2013-12-13 00:00:00|                1|       COMPLETE|
|   67863|2013-11-30 00:00:00|                2|       COMPLETE|
|   33865|2014-02-18 00:00:00|                2|       COMPLETE|
|   57963|2013-08-02 00:00:00|                2|        ON_HOLD|
|   15192|2013-10-29 00:00:00|                2|PENDING_PAYMENT|
|   23662|2013-12-19 00:00:00|                3|       COMPLETE|
|   22646|2013-12-11 00:00:00|                3|       COMPLETE|
|   57617|2014-07-24 00:00:00|                3|       COMPLETE|
|   35158|2014-02-26 00:00:00|                3|       COMPLETE|
|   61453|2013-12-14 00:00:00|                3|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 10 rows



## doing asceding and descing on columns

In [62]:
df.sort(col("order_customer_id"),col("order_status"),ascending=[0,1]).show(10)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|   61629|2013-12-21 00:00:00|            12435|       CANCELED|
|   41643|2014-04-08 00:00:00|            12435|        PENDING|
|    1868|2013-08-03 00:00:00|            12434|         CLOSED|
|    6160|2013-09-02 00:00:00|            12434|       COMPLETE|
|   61777|2013-12-26 00:00:00|            12434|       COMPLETE|
|   42915|2014-04-16 00:00:00|            12434|       COMPLETE|
|   51800|2014-06-14 00:00:00|            12434|        ON_HOLD|
|    5303|2013-08-26 00:00:00|            12434|        PENDING|
|   13544|2013-10-16 00:00:00|            12434|        PENDING|
|    4799|2013-08-23 00:00:00|            12434|PENDING_PAYMENT|
+--------+-------------------+-----------------+---------------+
only showing top 10 rows



## Sort Within Partitions

In [65]:
dt=[("a",1),("b",2),("c",3),("d",4),("e",5)]
dd=spark.createDataFrame(dt,schema=("col1","col2"))
dd.show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   c|   3|
|   d|   4|
|   e|   5|
+----+----+



## get the num of partionss

In [66]:
dd.rdd.getNumPartitions()

4

### check how data is distributed in these partitions

In [67]:
dd.rdd.glom().collect()

[[Row(col1='a', col2=1)],
 [Row(col1='b', col2=2)],
 [Row(col1='c', col2=3)],
 [Row(col1='d', col2=4), Row(col1='e', col2=5)]]

## here
* in partion 1 we have [Row(col1='a', col2=1)]
* similarly in Partiotn 4 we have [Row(col1='d', col2=4), Row(col1='e', col2=5)]]

## now if we use sort() it sort globally i.e alpplied on whole

## if we use sort Within partion

In [71]:
dd.sortWithinPartitions(dd.col1.desc()).show()
## check only partion is applied soed so we are getting "e" followed by "d"

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   c|   3|
|   e|   5|
|   d|   4|
+----+----+



# Set

In [4]:
df1=spark.range(10)
df1.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [5]:
df2=spark.range(5,15)
df2.show()

+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



In [6]:
df1.union(df2).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



In [6]:
df1=spark.createDataFrame(data=[("a",1),("b",2)],schema=("col1 string,col2 int"))
df2=spark.createDataFrame(data=[("b",3),("d",4)],schema=("col2 string,col1 int"))
print(df1.show())
print("-"*50)
print(df2.show())

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
+----+----+

None
--------------------------------------------------
+----+----+
|col2|col1|
+----+----+
|   b|   3|
|   d|   4|
+----+----+

None


In [12]:
## if we do union
df1.union(df2).show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   b|   3|
|   d|   4|
+----+----+



In [13]:
## if we do unionByName we get
df1.unionByName(df2).show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   3|   b|
|   4|   d|
+----+----+



## Intersect

In [15]:
df1=spark.createDataFrame(data=[("a",1),("a",1),("b",2)],schema=("col1 string,col2 int"))
df2=spark.createDataFrame(data=[("a",1),("a",1),("c",4)],schema=("col2 string,col1 int"))
df1.intersect(df2).show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
+----+----+



In [16]:
df1.intersectAll(df2).show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   a|   1|
+----+----+



## Except

In [17]:
df1.exceptAll(df2).show()

+----+----+
|col1|col2|
+----+----+
|   b|   2|
+----+----+



In [18]:
df2.exceptAll(df1).show()

+----+----+
|col2|col1|
+----+----+
|   c|   4|
+----+----+



## Join

In [3]:
df1=spark.createDataFrame([(1,"Robert"),(2,"julie"),(3,"James")],schema=["empID","empName"])
df2=spark.createDataFrame([(2,"USA"),(4,"India")],schema=["empID","Country"])
df2.show()

+-----+-------+
|empID|Country|
+-----+-------+
|    2|    USA|
|    4|  India|
+-----+-------+



### Inner_Join

In [4]:
df1.join(df2,on="empID",how="inner").show()

+-----+-------+-------+
|empID|empName|Country|
+-----+-------+-------+
|    2|  julie|    USA|
+-----+-------+-------+



In [5]:
df1.join(df2,df1.empID==df2.empID).show()

+-----+-------+-----+-------+
|empID|empName|empID|Country|
+-----+-------+-----+-------+
|    2|  julie|    2|    USA|
+-----+-------+-----+-------+



In [6]:
df1=spark.createDataFrame([(1,"Robert"),(2,"julie"),(3,"James")],schema=["empID","empName"])
df2=spark.createDataFrame([(2,"USA"),(4,"India")],schema=["ID","Country"])
df2.show()

+---+-------+
| ID|Country|
+---+-------+
|  2|    USA|
|  4|  India|
+---+-------+



In [10]:
df1.join(df2,df1.empID==df2.ID,how="inner").show()

+-----+-------+---+-------+
|empID|empName| ID|Country|
+-----+-------+---+-------+
|    2|  julie|  2|    USA|
+-----+-------+---+-------+



In [11]:
df1.join(df2,df1.empID==df2.ID,how="inner").select("ID","empID").show()

+---+-----+
| ID|empID|
+---+-----+
|  2|    2|
+---+-----+



## Left join

In [15]:
df1=spark.createDataFrame([(1,"Robert"),(2,"julie"),(3,"James")],schema=["empID","empName"])
df2=spark.createDataFrame([(2,"USA"),(4,"India")],schema=["empID","Country"])
df1.join(df2,df1.empID==df2.empID,how="left").show()

+-----+-------+-----+-------+
|empID|empName|empID|Country|
+-----+-------+-----+-------+
|    1| Robert| null|   null|
|    2|  julie|    2|    USA|
|    3|  James| null|   null|
+-----+-------+-----+-------+



In [19]:
df1.join(df2,df1.empID==df2.empID,how="left").select(df1.empID,"Country").show()

+-----+-------+
|empID|Country|
+-----+-------+
|    1|   null|
|    2|    USA|
|    3|   null|
+-----+-------+



## Right

In [22]:
df1.join(df2,df1.empID==df2.empID,how="right").select(df1.empID,"Country").show()

+-----+-------+
|empID|Country|
+-----+-------+
|    2|    USA|
| null|  India|
+-----+-------+



## Full===(Left Join + Right Join)

In [23]:
df1.join(df2,df1.empID==df2.empID,how="full").show()

+-----+-------+-----+-------+
|empID|empName|empID|Country|
+-----+-------+-----+-------+
|    1| Robert| null|   null|
|    2|  julie|    2|    USA|
|    3|  James| null|   null|
| null|   null|    4|  India|
+-----+-------+-----+-------+



## Cross Join---> For Cartesian product

In [24]:
df1.join(df2,df1.empID==df2.empID,how="cross").show()

+-----+-------+-----+-------+
|empID|empName|empID|Country|
+-----+-------+-----+-------+
|    2|  julie|    2|    USA|
+-----+-------+-----+-------+



## Left Anti join--- Only those recored will be the output ehich were not present in second dataset

In [25]:
df1.join(df2,df1.empID==df2.empID,how="leftanti").show()

+-----+-------+
|empID|empName|
+-----+-------+
|    1| Robert|
|    3|  James|
+-----+-------+



## LeftSemi-- Similar to inner join, only diff is that we cant select column from the second table in select statement

In [28]:
df1.join(df2,df1.empID==df2.empID,how="leftsemi").show()

+-----+-------+
|empID|empName|
+-----+-------+
|    2|  julie|
+-----+-------+



In [30]:
df1.join(df2,df1.empID==df2.empID,how="leftsemi").select(df1.empID,df2.Country).show()

AnalysisException: Resolved attribute(s) Country#170 missing from empID#165L,empName#166 in operator !Project [empID#165L, Country#170].;
!Project [empID#165L, Country#170]
+- Join LeftSemi, (empID#165L = empID#169L)
   :- LogicalRDD [empID#165L, empName#166], false
   +- LogicalRDD [empID#169L, Country#170], false


## Self Join

In [13]:
df=spark.createDataFrame([(1,"Robert",2),(2,"Jenny",3),(3,"James",5)],schema="empID int,empName string,managerid int")
df.show()

+-----+-------+---------+
|empID|empName|managerid|
+-----+-------+---------+
|    1| Robert|        2|
|    2|  Jenny|        3|
|    3|  James|        5|
+-----+-------+---------+



In [23]:
df.alias("d1").join(df.alias("d2"),col("d1.managerid")==col("d2.empID"),"left").select(col("d1.*"),\
                                                                                       col("d2.empName").alias("Manager_name")).show()

+-----+-------+---------+------------+
|empID|empName|managerid|Manager_name|
+-----+-------+---------+------------+
|    1| Robert|        2|       Jenny|
|    2|  Jenny|        3|       James|
|    3|  James|        5|        null|
+-----+-------+---------+------------+



## Joining using multiple variables(composite variables)

In [2]:
df1=spark.createDataFrame([(1,101,"Robert"),(2,102,"julie"),(3,103,"James")],schema="empID int,deptid int,empName string")
df2=spark.createDataFrame([(2,102,"USA"),(4,104,"India")],schema="empID int,deptid int,empName string")
df1.show()

+-----+------+-------+
|empID|deptid|empName|
+-----+------+-------+
|    1|   101| Robert|
|    2|   102|  julie|
|    3|   103|  James|
+-----+------+-------+



In [3]:
df1.join(df2,(df1.empID==df2.empID) & (df1.deptid==df2.deptid),how="inner").show()

+-----+------+-------+-----+------+-------+
|empID|deptid|empName|empID|deptid|empName|
+-----+------+-------+-----+------+-------+
|    2|   102|  julie|    2|   102|    USA|
+-----+------+-------+-----+------+-------+



## Multiframe dataframe join

In [6]:
df1=spark.createDataFrame([(1,"Robert"),(2,"Jenny"),(3,"James")],schema="empID int,empName string")
df2=spark.createDataFrame([(2,"USA"),(4,"India")],schema="empID int,country string")
df3=spark.createDataFrame([(1,"01-jan-2021"),(2,"01-feb-2021"),(3,"01-mar-2021")],schema="empID int,joindate string")
df2.show()

+-----+-------+
|empID|country|
+-----+-------+
|    2|    USA|
|    4|  India|
+-----+-------+



In [7]:
df1.join(df2,df1.empID==df2.empID,how="inner").join(df3,df1.empID==df3.empID,how="inner").show()

+-----+-------+-----+-------+-----+-----------+
|empID|empName|empID|country|empID|   joindate|
+-----+-------+-----+-------+-----+-----------+
|    2|  Jenny|    2|    USA|    2|01-feb-2021|
+-----+-------+-----+-------+-----+-----------+



## Aggregate

In [17]:
df=spark.read.load('Order_items/part-00000',format="csv",schema='order_id int,order_item_order_id int,order_item_product_id int, \
                   quantity int, subtotal float, price float')
df.show(3)

+--------+-------------------+---------------------+--------+--------+------+
|order_id|order_item_order_id|order_item_product_id|quantity|subtotal| price|
+--------+-------------------+---------------------+--------+--------+------+
|       1|                  1|                  957|       1|  299.98|299.98|
|       2|                  2|                 1073|       1|  199.99|199.99|
|       3|                  2|                  502|       5|   250.0|  50.0|
+--------+-------------------+---------------------+--------+--------+------+
only showing top 3 rows



In [23]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_item_order_id: integer (nullable = true)
 |-- order_item_product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- subtotal: float (nullable = true)
 |-- price: float (nullable = true)



### Summary

In [18]:
df.summary().show()

+-------+-----------------+-------------------+---------------------+------------------+------------------+------------------+
|summary|         order_id|order_item_order_id|order_item_product_id|          quantity|          subtotal|             price|
+-------+-----------------+-------------------+---------------------+------------------+------------------+------------------+
|  count|           172198|             172198|               172198|            172198|            172198|            172198|
|   mean|          86099.5|  34442.56682423721|    660.4877176273824|2.1821275508426345|199.32066922046081|133.75906959048717|
| stddev|49709.42516431533| 19883.325171992343|     310.514472790008|1.4663523175387134|112.74303987146804|118.55893633258484|
|    min|                1|                  1|                   19|                 1|              9.99|              9.99|
|    25%|            43033|              17204|                  403|                 1|            119.98|    

In [20]:
df.summary().select("summary","price","subtotal").show()

+-------+------------------+------------------+
|summary|             price|          subtotal|
+-------+------------------+------------------+
|  count|            172198|            172198|
|   mean|133.75906959048717|199.32066922046081|
| stddev|118.55893633258484|112.74303987146804|
|    min|              9.99|              9.99|
|    25%|              50.0|            119.98|
|    50%|             59.99|            199.92|
|    75%|            199.99|            299.95|
|    max|           1999.99|           1999.99|
+-------+------------------+------------------+



## Sum and sumDistinct

In [30]:
df.select(round(sum("price"),2).alias("Sum_price"),round(sumDistinct("price"),2).alias("Sum_Distinct")).show()

D:\Software_install\Anaconda\envs\SPARK_HOME\lib\site-packages\pyspark\sql\functions.py:315: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+-------------+------------+
|    Sum_price|Sum_Distinct|
+-------------+------------+
|2.303304427E7|     9832.42|
+-------------+------------+



## Count and CountDistinct

In [33]:
df.select(count("price").alias("Count_price"),countDistinct("price").alias("Count_Distinct")).show()

+-----------+--------------+
|Count_price|Count_Distinct|
+-----------+--------------+
|     172198|            57|
+-----------+--------------+



In [38]:
df.select(max("price")).show()

+----------+
|max(price)|
+----------+
|   1999.99|
+----------+



In [40]:
df.select(min("price")).show()

+----------+
|min(price)|
+----------+
|      9.99|
+----------+



## collect_list and collect_set

In [41]:
df=spark.createDataFrame([(1,100),(2,150),(3,200),(4,50),(5,500)],schema="id int,salary int")
df.show()

+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   150|
|  3|   200|
|  4|    50|
|  5|   500|
+---+------+



In [42]:
from pyspark.sql.functions import collect_list,collect_set

In [45]:
df.select(collect_list("salary").alias("list"),collect_set("salary").alias("set")).show(truncate=False)

+------------------------+------------------------+
|list                    |set                     |
+------------------------+------------------------+
|[100, 150, 200, 50, 500]|[150, 100, 50, 500, 200]|
+------------------------+------------------------+



### here
* list contain all salary and set contain distinct salary